# Southern California Air Quality Bot

This is a notebook for downloading air quality data from [PurpleAir](https://purpleair.com)
and broadcasting it on social media. If the air quality is *bad*, it can also tag in
relevant elected officials.

In [ ]:
import os
import re
from typing import Optional

import contextily
import dotenv
import geopandas
import matplotlib.pyplot as plt
import numpy
import pandas
import requests
from matplotlib.colors import LinearSegmentedColormap
import tweepy

dotenv.load_dotenv()

## Download the data

We first download the data from Purple Air.

In [ ]:
API_KEY = os.environ.get("PURPLEAIR_API_KEY")
FIELDS = [
    "name",
    "location_type",
    "latitude",
    "longitude",
    "humidity",
    "pm2.5_cf_1",
    "pm2.5_cf_1_a",
    "pm2.5_cf_1_b",

]
response = requests.get(
    "https://api.purpleair.com/v1/sensors",
    headers={"X-API-Key": API_KEY},
    params={
        "fields": ",".join(FIELDS),
        "max_age": 10*60,
        "nwlng": -119.4,
        "nwlat": 35.0,
        "selng": -116.6,
        "selat": 32.5,
    }   
)
response.raise_for_status()
data = response.json()
df = pandas.DataFrame.from_records(data=data["data"], columns=data["fields"])

In [ ]:
gdf = geopandas.GeoDataFrame(
    df,
    geometry=geopandas.points_from_xy(df["longitude"], df["latitude"], crs="EPSG:4326")
)
# Filter to outdoors and recently-seen sensors. Age seems to be in minutes
gdf = gdf[(gdf.location_type == 0)]

# Remove extreme outliers based on the difference between the two channels
gdf = gdf[(gdf["pm2.5_cf_1_a"] - gdf["pm2.5_cf_1_b"]).abs() < 50]

## Compute AQI from the data

[airnow.gov](airnow.gov) has an PM2.5-to-AQI [calculator](https://www.airnow.gov/aqi/aqi-calculator-concentration/)
which [exposes](https://www.airnow.gov/sites/default/files/custom-js/conc-aqi.js) some human-readable javascript utils.
These are ported to Python here:

In [ ]:
def aqi_from_pm(pm: float) -> Optional[float]:
    # Check for invalid readings. PM > 500 is off the scale.
    if pandas.isna(pm) or pm < 0 or pm >= 500.5:
        return None; 

    if pm > 0 and pm < 12.1:
        return linear(50, 0, 12, 0, pm)
    elif pm >= 12.1 and pm < 35.5:
        return linear(100, 51, 35.4, 12.1, pm)
    elif pm >= 35.5 and pm < 55.5:
        return linear(150, 101, 55.4, 35.5, pm)
    elif pm >= 55.5 and  pm < 150.5:
        return linear(200, 151, 150.4, 55.5, pm)
    elif pm >= 150.5 and pm < 250.5:
        return linear(300, 201, 250.4, 150.5, pm)
    elif pm >= 250.5 and pm < 350.5:
        return linear(400, 301, 350.4, 250.5, pm)
    elif pm >= 350.5 and pm < 500.5:
        return linear(500, 401, 500.4, 350.5, pm)
    else:
        return None

def linear(aqi_high, aqi_low, conc_high, conc_low, conc):
    return numpy.rint(
        ((aqi_high - aqi_low) / (conc_high - conc_low)) * (conc - conc_low) + aqi_low
    )

We also apply the EPA correction to raw Purple Air sensor data described
[here](https://cfpub.epa.gov/si/si_public_record_Report.cfm?dirEntryId=350075&Lab=CEMM),
which was calibrated to wildfire data in 2020 and removes some apparent systematic over-estimates.

In [ ]:
# Calculate the AQI values and drop nulls.
gdf = (
    gdf
    .assign(
        aqi=(0.52 * gdf["pm2.5_cf_1"] - 0.085 * gdf.humidity + 5.71).apply(aqi_from_pm)
    )
    .dropna(subset=["aqi"])
)

## Create plots!

Administrative districts, especially LA City Council districts, can have weird shapes
due to historical contingencies and gerrymandering. This proved to have some odd effects
when doing simple point-in-polygon tests for sensors, as sensors which have relevant
data to a region were being cut out due to a weird wiggle in a boundary.

Instead, I create a buffered convex hull for each district and use that for filtering
to relevant sensor data.

In [ ]:
if os.environ.get("DISTRICT_TYPE") == "SUPERVISORIAL":
    kind = "LA Supervisorial District"
else:
    kind = "LA City Council District"

In [ ]:
districts = geopandas.read_file("data/districts.geojson")
districts = districts[districts.kind == kind]

# Create an envelope with a 1 mile buffer
districts = (
    districts
    .to_crs(epsg=2229)
    .assign(envelope=lambda gdf: gdf.geometry.convex_hull.buffer(5280.))
    .set_geometry("envelope")
    .to_crs(epsg=4326)
    .set_geometry("geometry")
)

# A fun plot of the districts!
districts.set_geometry("envelope").plot(alpha=0.5)

In [ ]:
cmap = LinearSegmentedColormap.from_list(
    name="purpleair",
    colors=["green", "yellow", "red", "purple"]
)

## Tweet things!

In [ ]:
def get_category(aqi: float) -> str:
    if aqi <=50:
        return "good"
    elif aqi > 50 and aqi <= 100:
        return "moderate"
    elif aqi > 100 and aqi <= 150:
        return "unhealthy for sensitive groups"
    elif aqi > 150 and aqi <= 200:
        return "unhealthy"
    elif aqi > 200 and aqi <= 300:
        return "very unhealthy"
    elif aqi > 300 and aqu <= 500:
        return "hazardous"
    else:
        return "out of range"

In [ ]:
# We'd prefer to use the v2 of the API with tweepy.Client, but that doesn't yet support
# uploading media, so fall back to v1.
auth = tweepy.OAuthHandler(
    consumer_key=os.environ.get("TWITTER_API_KEY"),
    consumer_secret=os.environ.get("TWITTER_API_KEY_SECRET"),
)
auth.set_access_token(
    os.environ.get("TWITTER_ACCESS_TOKEN"),
    os.environ.get("TWITTER_ACCESS_TOKEN_SECRET"),
)
client = tweepy.API(auth) 

Okay, we want to send tweets over the course of each day, rather than a bulk dump of them.
This is a bit tricky with GitHub actions, as they are designed to be somewhat stateless,
so it's not easy to know which row to tweet about.

So here we use Twitter itself as our state store: we grab the most recent tweet
from the account, see what it was, then tweet for the next district. This is brittle,
and will likely need to be rethought if this ever extends beyond city council/supervisorial districts!

In [ ]:
tweets = client.user_timeline(count=20)

if kind == "LA City Council District":
    size=4000
    alpha=0.7
    n_districts = 15
    try:
        for t in tweets:
            matches = re.findall(r"LA City Council District (\d+)", t.text)
            if len(matches):
                next_district = int(matches[0]) + 1
                next_district = next_district if next_district <= n_districts else 1
                break
        else:
            next_district = 1 
    except:
        next_district = 1
elif kind == "LA Supervisorial District":
    size=1000
    alpha=0.3
    n_districts = 5
    try:
        for t in tweets:
            matches = re.findall(r"LA Supervisorial District (\d+)", t.text)
            if len(matches):
                next_district = int(matches[0]) + 1
                next_district = next_district if next_district <= n_districts else 1
                break
        else:
            next_district = 1 
    except:
        next_district = 1

districts_to_tweet = [next_district]

In [ ]:
for i, district in districts.iterrows():
    if district.district not in districts_to_tweet:
        continue
    district_gdf = gdf[gdf.geometry.within(district.envelope)]
    
    # Create text for a tweet
    aqi = int(numpy.rint(district_gdf.aqi.median()))
    category = get_category(aqi)
    message = f"""The residents of {district.label} are experiencing an air quality index of {aqi} ({category})."""
    if aqi > 100:
        if pandas.notna(district.handle):
            message += f"\n\n{district.handle} what are you doing to make sure that the people you represent have safe air to breathe?"

    # Set up the figure
    fig, ax = plt.subplots(figsize=(16,16))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    # Plot the outline
    districts[
        districts.district == district.district
    ].to_crs(epsg=3857).plot(ax=ax, color="blue", alpha=0.1)
    
    # Plot the data
    ax.axes.autoscale(False)
    
    district_gdf.to_crs(epsg=3857).plot(ax=ax, s=size, alpha=alpha, column="aqi", cmap=cmap, vmin=0, vmax=200)
    
    # Annotations are too busy for supervisorial districts
    if kind == "LA City Council District" or True:
        for idx, sensor in district_gdf.to_crs(epsg=3857).iterrows():
            color = "black" if sensor["aqi"] < 120 else "white"
            ax.annotate(
                str(int(sensor["aqi"])),
                xy=(sensor.geometry.x, sensor.geometry.y),
                fontsize=16,
                ha="center",
                va="center",
                color=color,
                alpha=alpha,
                annotation_clip=True,
                clip_on=True,
            )
        
    # Add the basemap
    contextily.add_basemap(ax)
    
    # Output
    if kind == "LA Supervisorial District":
        filename = f"sd{district.district}.png"
    elif kind == "LA City Council District":
        filename = f"cd{district.district}.png"
    plt.close(fig)
    fig.savefig(filename, bbox_inches="tight")
    
    # Send tweet
    media = client.media_upload(filename)
    client.update_status(
        status=message,
        media_ids=[media.media_id],
        lat=district.latitude,
        long=district.longitude,
        display_coordinates=True,
    )